# STEP 1: Import FaaSET

In [3]:
import FaaSET

------------------------------------------------------
Welcome to FaaSET v2.0! Some things have changed when
compared to v1.0. RunModes and the Containerize arguments
have been removed in favor of more available Platforms.
Platforms is no longer a list and now functions can
only be deployed to a single platform at a time.
If you are using a older notebook that has not been
updated things will be broken. Please fix them. 
------------------------------------------------------
Loaded platforms:  AWS. IBM Docker. AWS Docker ARM. OpenFaaS EKS. GCF Gen2. OpenFaaS. IBM. AWS Docker. AWS Docker Debian. Azure. Local. GCF. 
------------------------------------------------------
Loaded functions:  hello_world_c. 


In [5]:
import pprint
import pandas as pd
import numpy as np
import time

pp = pprint.PrettyPrinter(indent=4)

# STEP 2: Create a dummy function. This will setup the folder that we will add files to in the next cells...

In [5]:
@FaaSET.cloud_function(platform="AWS Docker Debian")
def hello_world_c(request, context): 
    return {"message": "Dummy Function"}   

hello_world_c({}, None)

{'message': 'Dummy Function'}

# STEP 3: Create the C file:

In [6]:
%%writefile functions/hello_world_c/hello_world.c

#include <stdio.h>

int main()
{
    printf("Dello");
    return 0;
}

Writing functions/hello_world_c/hello_world.c


# STEP 4: Update the Dockerfile to compile the C:

In [5]:
%%writefile functions/hello_world_c/Dockerfile

#FROM python:buster as build-image
FROM python:3.8-slim-buster

# Update and install curl. 
RUN apt update
RUN apt install -y curl build-essential 

# Define and create function directory
ENV FUNCTION_DIR="/var/task"
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY . ${FUNCTION_DIR}

RUN pip install boto3 botocore

# Install the runtime interface client
RUN pip install \
        --target ${FUNCTION_DIR} \
        awslambdaric

# Install the runtime interface emulator
RUN curl -Lo /usr/local/bin/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie
RUN chmod +x /usr/local/bin/aws-lambda-rie

# Set working directory
WORKDIR ${FUNCTION_DIR}

# Compile C code
RUN gcc -o stream.10M -O -DSTREAM_ARRAY_SIZE=10000000 -DNTIMES=10 stream_small.c


# Set entry script that determines whether to run RIE if running locally
COPY ./entry_script.sh /entry_script.sh
RUN chmod +x /entry_script.sh
ENTRYPOINT [ "/entry_script.sh" ]

# Set handler
CMD [ "lambda_function.lambda_handler" ]

Overwriting functions/hello_world_c/Dockerfile


# Step 5: Create your real function that will shell out and run the C:

In [17]:
# @FaaSET.cloud_function(platform="AWS Docker Debian", config={"memory": 259, "profile": "personal"})
@FaaSET.cloud_function(platform="AWS Docker Debian", config={"memory": 259})
def hello_world_c(request, context): 
    import subprocess

    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
    command = "./stream.10M"
    metric_outputs = list()
    for x in range(20):
        output = subprocess.check_output(command.split()).decode('ascii')
        data = output.split('\n')[23:27]
        metric = list(map(lambda i: list(map(float, i.split()[1:])), data))
        metric_outputs.append(metric)
    inspector.addAttribute("stream_metric", metric_outputs)
    inspector.inspectAllDeltas()
    return inspector.finish()
#     return {"message": output}   

# hello_world_c({}, None)

In [ ]:
metric

In [18]:
%time ret = hello_world_c({}, None)

Build: Loading config...
Build: Purging previous build...
Build: Copying files...
Build: Cleaning up...
Build: Building Docker Image...
Sending build context to Docker daemon  64.51kB

Step 1/16 : FROM python:3.8-slim-buster
 ---> c706fbd78639
Step 2/16 : RUN apt update
 ---> Using cache
 ---> 9e15b7761466
Step 3/16 : RUN apt install -y curl build-essential
 ---> Using cache
 ---> d20ecd952a4a
Step 4/16 : ENV FUNCTION_DIR="/var/task"
 ---> Using cache
 ---> 8c2971bc0243
Step 5/16 : RUN mkdir -p ${FUNCTION_DIR}
 ---> Using cache
 ---> ec6f531b3418
Step 6/16 : COPY . ${FUNCTION_DIR}
 ---> f2bbb74b5356
Step 7/16 : RUN pip install boto3 botocore
 ---> Running in a15ca16c5b2a
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 22.9 MB/s eta 0:00:0

 ---> bf1f7d228777
Step 7/16 : RUN pip install boto3 botocore
 ---> Running in 5ee8774d3541
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 22.8 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Removing intermediate container 5ee8774d3541
 ---> b9e4fe0c3145
Step 8/16 : RUN pip install         --target ${FUNCTION_DIR}         awslambdaric
 ---> Running in dfd217a174cf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 KB 18.9 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip

========================

In [17]:
from multiprocessing.pool import Pool

In [1]:
import requests
import json

In [19]:
%time ret = requests.get('https://j4fohrjqqyhud7giienbqoozxa0jrbyj.lambda-url.us-east-2.on.aws/')

CPU times: user 23.2 ms, sys: 7.63 ms, total: 30.8 ms
Wall time: 18.4 s


In [2]:
def run_function(url = 'https://j4fohrjqqyhud7giienbqoozxa0jrbyj.lambda-url.us-east-2.on.aws/'):
        ret = requests.get(url)
        return json.loads(ret.text)

## run 1 time

In [12]:
%%time
results = []
for i in range(50):
    ret = run_function()
    results.append(ret)
    

CPU times: user 1.22 s, sys: 139 ms, total: 1.36 s
Wall time: 14min 17s


In [13]:
result_df = pd.DataFrame(results)

In [14]:
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/1/', result_df, name='2', con=1)

In [18]:
def run_exp(con=50):
    p = Pool(con)
    result = p.map(run_function, ['https://j4fohrjqqyhud7giienbqoozxa0jrbyj.lambda-url.us-east-2.on.aws/'] * con)
    return result

In [19]:
%time result1 = run_exp(con=100)

CPU times: user 0 ns, sys: 1.74 s, total: 1.74 s
Wall time: 21.4 s


In [20]:
%time result2 = run_exp(con=100)

CPU times: user 0 ns, sys: 1.73 s, total: 1.73 s
Wall time: 20.4 s


In [21]:
%time result3 = run_exp(con=100)

CPU times: user 0 ns, sys: 1.86 s, total: 1.86 s
Wall time: 20.5 s


In [22]:
%time result4 = run_exp(con=100)

CPU times: user 17.6 ms, sys: 1.89 s, total: 1.91 s
Wall time: 21.2 s


In [23]:
%time result5 = run_exp(con=100)

CPU times: user 144 ms, sys: 1.69 s, total: 1.84 s
Wall time: 20 s


In [35]:
df = pd.DataFrame(result5)

In [24]:
def parse_data_and_write(filepath, results, name=None, con=100):
    if name is None:
        name = str(int(time.time()))
#     result = run_exp(con=con)
    result_df = pd.DataFrame(results)
#     result_df.to_csv('~/stream_metric_100.csv')
    result_df.to_csv(filepath + '/stream_raw_{}_{}.csv'.format(con, name))
    result_df.sort_values('initializationTime', inplace=True)
    assert result_df['endTime'].min() - result_df['initializationTime'].max() > 0, 'not all funcs all at the same time!'
    uuid_group = result_df.groupby('uuid')
    
    assert uuid_group.count().shape[0] == con, 'uuid show duplicates'
#     print()
    arr = result_df['stream_metric'].to_numpy()
    arr = np.stack(arr, axis=0)
#     avg_arr = arr[:,:, 0]
#     result_df['stream_copy_avg'] = avg_arr[:, 0]
#     result_df['stream_scale_avg'] = avg_arr[:, 1]
#     result_df['stream_add_avg'] = avg_arr[:, 2]
#     result_df['stream_triad_avg'] = avg_arr[:, 3]

    result_df['stream_copy_avg'] = np.average(arr[:, :, 0, 0], axis=1)
    result_df['stream_scale_avg'] = np.average(arr[:, :, 1, 0], axis=1)
    result_df['stream_add_avg'] = np.average(arr[:, :, 2, 0], axis=1)
    result_df['stream_triad_avg'] = np.average(arr[:, :, 3, 0], axis=1)
    
    metric_df = result_df[['uuid', 'stream_copy_avg', 'stream_scale_avg', 'stream_add_avg', 'stream_triad_avg', 'runtime', 'pageFaultsDelta']]
    metric_df.to_csv(filepath + '/stream_metric_{}_{}.csv'.format(con, name))

In [25]:
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/', result1, name='1')
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/', result2, name='2')
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/', result3, name='3')
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/', result4, name='4')
parse_data_and_write('/home/itadmin/exp_data/stream/5.18/', result5, name='5')


In [26]:
%%time
result_50_con = list()

for i in range(16):
    tmp_result = run_exp(50)
    
for i in range(4):
    tmp_result = run_exp(50)
    result_50_con.append(tmp_result)

CPU times: user 3.94 s, sys: 15.6 s, total: 19.6 s
Wall time: 6min 28s


In [27]:
%%time 
for i in range(4):
    parse_data_and_write('/home/itadmin/exp_data/stream/5.18/50', result_50_con[i], name='{}'.format(i), con=50)

CPU times: user 247 ms, sys: 23.6 ms, total: 271 ms
Wall time: 266 ms


In [28]:
%%time
result_25_con = list()

for i in range(12):
    tmp_result = run_exp(25)

for i in range(8):
    tmp_result = run_exp(25)
    result_25_con.append(tmp_result)

CPU times: user 1.97 s, sys: 7.88 s, total: 9.85 s
Wall time: 6min 16s


In [29]:
%%time 
for i in range(8):
    parse_data_and_write('/home/itadmin/exp_data/stream/5.18/25', result_25_con[i], name='{}'.format(i), con=25)

CPU times: user 359 ms, sys: 0 ns, total: 359 ms
Wall time: 353 ms


In [30]:
%%time
result_10_con = list()

for i in range(20):
    tmp_result = run_exp(10)
    result_10_con.append(tmp_result)

CPU times: user 649 ms, sys: 3.46 s, total: 4.11 s
Wall time: 6min


In [31]:
%%time 
for i in range(20):
    parse_data_and_write('/home/itadmin/exp_data/stream/5.18/10', result_10_con[i], name='{}'.format(i), con=10)

CPU times: user 615 ms, sys: 0 ns, total: 615 ms
Wall time: 609 ms
